<a href="https://colab.research.google.com/github/purushottampanchal/temp/blob/main/tortoise_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [1]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 19.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (408/408), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1481 (delta 364), reused 359 (delta 359), pack-reused 1073
Receiving objects: 100% (1481/1481), 53.55 MiB | 15.34 MiB/s, done.
Resolving deltas: 100% (615/615), done.
/content/tortoise-tts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00
 11% (11476992 of 100715777) |#          | Elapsed Time: 0:00:00 ETA:  00:00:00

Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


In [15]:
# This is the text that will be spoken.
text = "Simply put, a paragraph is a collection of sentences all related to a central topic, idea, or theme. Paragraphs act as structural tools for writers to organize their thoughts into an ideal progression, and they also help readers process those thoughts effortlessly. Imagine how much harder reading and writing would be if everything was just one long block of text."

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

In [5]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!cp /content/drive/MyDrive/00089001.wav /content/tortoise-tts/tortoise/voices/Custom/1.wav
!cp /content/drive/MyDrive/00095001.wav /content/tortoise-tts/tortoise/voices/Custom/2.wav
!cp /content/drive/MyDrive/00106001.wav /content/tortoise-tts/tortoise/voices/Custom/3.wav


In [10]:
# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.

CUSTOM_VOICE_NAME = "Custom"

##dont run below code
# import os
# from google.colab import files

# custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
# os.makedirs(custom_voice_folder)
# for i, file_data in enumerate(files.upload().values()):
#   with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
#     f.write(file_data)

In [14]:
IPython.display.Audio(f'/content/tortoise-tts/tortoise/voices/Custom/1.wav')

In [16]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [05:39<00:00, 21.22s/it]


Computing best candidates using CLVP and CVVP


  6%|▋         | 1/16 [00:01<00:23,  1.53s/it]

No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.


...

 94%|█████████▍| 15/16 [00:25<00:01,  1.70s/it]

No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.
No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.


100%|██████████| 16/16 [00:26<00:00,  1.69s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [08:43<00:00,  1.31s/it]
